In [16]:
import numpy as np
import os
import pickle
import torch
from typing import List, Dict, Tuple
import pandas as pd
import torch.nn.functional as F
from genReLiTu import plot_detection_heatmaps_3x4, generate_data_1dimension
import sklearn.metrics.pairwise as smp
from collections import defaultdict
from sklearn.cluster import KMeans

In [17]:
dataset = "fmnist"
attack_defense_data = {
    "NEUROTOXIN": {
        "AVG": "../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg_NEUROTOXIN-fmnist/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_01-57-26/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_12-39-47/",
    },
    "MR": {
        "AVG": "FL_Backdoor_CV/2024-09-11_16-29-41/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_05-15-58/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_06-21-04/",
    },
    "EDGE_CASE": {
        "AVG": "FL_Backdoor_CV/2024-09-11_17-32-14/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_10-44-13/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_13-45-18/",
    },
}

In [18]:
# for attack, attack2data in attack_defense_data.items():
#     true_labels = []
#     labels_get_by_Euclid = []
#     labels_get_by_manhattan = []
#     labels_get_by_cosine = []
#     labels_get_by_chi_square = []
#     for denfense, data_folder in attack2data.items():
#         if denfense != "AVG":
#             continue
#         participant_file_name = f"{data_folder}participants/participants.csv"
#         participants = np.genfromtxt(
#             participant_file_name, delimiter=",", dtype=None, encoding="utf-8"
#         )
#         participants = participants[1:].T
#         for i in range(len(participants) - 1):
#             file_name = f"{data_folder}model_updates/{dataset}_{attack}_{i}.pkl"
#             if not os.path.exists(file_name):
#                 print(f"File {file_name} not found")
#                 continue
#             with open(file_name, "rb") as file:
#                 model_updates = pickle.load(file)


# roundNum = 10
# dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg_NEUROTOXIN-fmnist/'
# pklName = os.path.join(dirPath, f'model_updates/fmnist_NEUROTOXIN_{roundNum}.pkl')
# participantFilePath = os.path.join(dirPath, 'participants/participants.csv')
# # participantFileDF = pd.read_csv(participantFilePath)
# # # 提取参与者数组（第一列）和轮次（从第二列开始的部分）
# # clients = participantFileDF.iloc[1:, 0].tolist()  # 客户端列表
# # roundsArray = participantFileDF.iloc[1:, 1:].to_numpy()  # 轮次数据（50行x30列）
# # print(roundsArray)
# participants = np.genfromtxt(
#     participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
# )
# participants = participants[1:].T
# participants_thisRound = participants[roundNum]
# print(participants_thisRound)

# with open(pklName, 'rb') as f:
#     update: Dict[str, torch.Tensor] = pickle.load(f)
# # print(update)
# # weight0 = update['base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight']
# # weight0.shape  # torch.Size([50, 12288])
# userUpdates = [torch.empty(0) for _ in range(50)]
# for layerKey, values in update.items():
#     # print(layerKey)
#     # print(values)
#     # print(values.shape)  # torch.Size([50, 12288])
#     for i in range(values.shape[0]):
#         trueUser = participants_thisRound[i]
#         userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i].cpu()), 0)
# print(userUpdates[0].shape)

def loadPkl(roundNum: int, dirPath: str) -> Dict[str, torch.Tensor]:  # 把pkl变成{'key1': [[user3], [user5]]}
    # dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    pklPrefix = os.path.basename(os.path.normpath(dirPath)).split('-')[-1]
    pklName = os.path.join(dirPath, f'model_updates/{pklPrefix}_{roundNum}.pkl')
    with open(pklName, 'rb') as f:
        update: Dict[str, torch.Tensor] = pickle.load(f)
    return update

def getParticipants(roundNum: int, dirPath: str) -> np.ndarray:  # 获取参与者列表
    participantFilePath = os.path.join(dirPath, 'participants/participants.csv')

    # 读取参与者数组
    participants = np.genfromtxt(
        participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
    )
    participants = participants[1:].T
    participants_thisRound = participants[roundNum]  # 获取当前轮次的参与者
    return participants_thisRound

def get_all_user_updates(roundNum: int, dirPath: str) -> torch.Tensor:  # 把pkl变成[[user1展平(拼接)后的结果], [user2], ...]
    # 设置轮次编号和文件路径
    # roundNum = 10
    # dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    participants_thisRound = getParticipants(roundNum, dirPath)
    # print(participants_thisRound)

    # 加载 .pkl 文件
    update = loadPkl(roundNum, dirPath)

    # 初始化一个列表来存储每个客户端的更新，使用 GPU 张量
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    userUpdates = [torch.empty((0,), device=device) for _ in range(50)]  # 假设50个客户端

    # 遍历每个层的更新
    for layerKey, values in update.items():
        # 将每个值移到 GPU 上
        values = values.to(device)
        for i in range(values.shape[0]):  # 遍历每个客户端的梯度
            trueUser = participants_thisRound[i]  # 获取当前轮次的客户端ID
            # 如果为空张量
            if userUpdates[trueUser].numel() == 0:
                userUpdates[trueUser] = values[i]  # 直接赋值
            else:
                userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i]), 0)  # 在GPU上拼接

    userUpdates = torch.stack(userUpdates)  # 转换为张量
    # # 打印第一个用户的更新形状来检查结果
    # print(userUpdates[0].shape)
    return userUpdates

In [19]:
def cosine_similarity_matrix(tensor: torch.Tensor) -> torch.Tensor:
    """
    计算一个二维 tensor 中每两个向量之间的余弦相似度矩阵。

    参数:
    - tensor (torch.Tensor): 形状为 (N, D) 的二维张量，N 是客户端数量，D 是特征维度。

    返回:
    - similarity_matrix (torch.Tensor): 形状为 (N, N) 的张量，每个元素表示两个向量之间的余弦相似度。
    """
    # 归一化每个向量
    normalized_tensor = F.normalize(tensor, p=2, dim=1)
    
    # 计算余弦相似度矩阵
    similarity_matrix = torch.mm(normalized_tensor, normalized_tensor.T)
    
    return similarity_matrix


In [20]:
# userUpdates_avg_NEUR = get_all_user_updates(10, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/')
# print(userUpdates_avg_NEUR.shape)
# print(userUpdates_avg_NEUR)
# userUpdates_avg_MR = get_all_user_updates(10, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-13_12-02-44-avg-fmnist_MR/')

# similarity_matrix_avg_NEUR = cosine_similarity_matrix(userUpdates_avg_NEUR)
# print(similarity_matrix_avg_NEUR.shape)
# similarity_matrix_avg_MR = cosine_similarity_matrix(userUpdates_avg_MR)
# # print(similarity_matrix_avg_MR.shape)
# plot_detection_heatmaps(similarity_matrix_avg_NEUR, similarity_matrix_avg_MR)

In [32]:
# foolsgold
# ChangeFrom https://github.com/LetMeFly666/SecFFT/blob/706bb287c3b00f6143e2190edc74714ed88f3532/getResult/FL_Backdoor_CV/roles/aggregation_rules.py#L203C1-L241C25
# foolsgold自身并没有确认哪些是恶意客户端，而是根据权重聚合
# def foolsgold_identify_malicious_clients(wv: np.ndarray, threshold: float = 0.25) -> np.ndarray:
#     """
#     根据权重向量识别潜在的恶意客户端。

#     参数:
#     - wv (np.ndarray): 客户端的权重向量。
#     - threshold (float): 判断恶意客户端的阈值，权重低于此值的客户端将被视为恶意客户端。
    
#     返回:
#     - malicious_clients (np.ndarray): 被识别为恶意的客户端索引。
#     """
#     return np.where(wv < threshold)[0]
def foolsgold_identify_malicious_clients(wv: np.ndarray) -> np.ndarray:
    kmeans = KMeans(n_clusters=2, random_state=0).fit(wv.reshape(-1, 1))
    clusters = kmeans.labels_
    unique, counts = np.unique(clusters, return_counts=True)
    max_cluster = unique[np.argmax(counts)]
    user_labels = np.zeros(len(clusters), dtype=int)
    user_labels[clusters == max_cluster] = 1
    return np.where(user_labels == 0)[0]



# 返回 聚合后的全局模型、恶意客户端索引、余弦相似度矩阵
# def foolsgold(model_updates: Dict[str, torch.Tensor]) -> Tuple[Dict[str, torch.Tensor], np.ndarray, np.ndarray]:
def foolsgold(roundNum: int, dirPath: str) -> Tuple[Dict[str, torch.Tensor], np.ndarray, torch.Tensor]:
    model_updates = loadPkl(roundNum, dirPath)
    participants_thisRound = getParticipants(roundNum, dirPath)
    
    keys = list(model_updates.keys())
    last_layer_updates = model_updates[keys[-2]]
    K = len(last_layer_updates)
    cs = smp.cosine_similarity(last_layer_updates.cpu().numpy()) - np.eye(K)  # 减去对角线为1的单位矩阵，使得自身与自身的相似度为0
    maxcs = np.max(cs, axis=1)
    # === pardoning(赦免) ===
    for i in range(K):
        for j in range(K):
            if i == j:
                continue
            if maxcs[i] < maxcs[j]:
                cs[i][j] = cs[i][j] * maxcs[i] / maxcs[j]

    alpha = np.max(cs, axis=1)
    wv = 1 - alpha
    wv[wv > 1] = 1
    wv[wv < 0] = 0

    # === Rescale so that max value is wv ===
    wv = wv / np.max(wv)
    wv[(wv == 1)] = .99

    # === Logit function ===
    wv = (np.log(wv / (1 - wv)) + 0.5)
    wv[(np.isinf(wv) + wv > 1)] = 1
    wv[(wv < 0)] = 0

    malicious_clients = foolsgold_identify_malicious_clients(wv)
    malicious_clients = participants_thisRound[malicious_clients]
    print(f"识别出的恶意客户端索引: {malicious_clients}")

    # === calculate global update ===
    global_update = defaultdict()
    for name in keys:
        tmp = None
        for i, j in enumerate(range(len(wv))):
            if i == 0:
                tmp = model_updates[name][j] * wv[j]
            else:
                tmp += model_updates[name][j] * wv[j]
        global_update[name] = 1 / len(wv) * tmp
    print(wv)
    if True:  # 使用余弦相似度作为热力图依据
        cs = smp.cosine_similarity(last_layer_updates.cpu().numpy())  # 这里就不再减去自身了
        cs_rearranged = np.zeros((len(participants_thisRound), len(participants_thisRound)))
        for i, user_i in enumerate(participants_thisRound):
            for j, user_j in enumerate(participants_thisRound):
                cs_rearranged[user_i][user_j] = cs[i][j]
        cs_tensor = torch.from_numpy(cs_rearranged)
    else:  # 使用聚合参数wv作为热力图依据。这样直接很多1，太明显了
        cs_tensor = torch.zeros((len(participants_thisRound)))
        for i, user_i in enumerate(participants_thisRound):
            cs_tensor[user_i] = wv[i]  # tensor([0.2348, 0.2348, 0.8740, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

    return global_update, malicious_clients, cs_tensor

In [40]:
_, foolsgoldMaliciousIndex, foolsgoldScore2 = foolsgold(19, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-13_23-15-48-foolsgold-fmnist_NEUROTOXIN')
print(foolsgoldMaliciousIndex)
print(foolsgoldScore2)
datas = [foolsgoldScore2] * 12
plot_detection_heatmaps_3x4(*datas)

识别出的恶意客户端索引: [0 1]
[1.         1.         1.         1.         0.23480443 1.
 1.         1.         0.23480443 0.87404559]
[0 1]
tensor([[1.0000, 0.9253, 0.9060, 0.7916, 0.7922, 0.7943, 0.7986, 0.7680, 0.8143,
         0.7611],
        [0.9253, 1.0000, 0.9115, 0.7851, 0.7924, 0.7995, 0.7941, 0.7534, 0.8105,
         0.7606],
        [0.9060, 0.9115, 1.0000, 0.8156, 0.8030, 0.8189, 0.8185, 0.7997, 0.8212,
         0.8038],
        [0.7916, 0.7851, 0.8156, 1.0000, 0.8738, 0.8763, 0.8671, 0.8386, 0.8235,
         0.8070],
        [0.7922, 0.7924, 0.8030, 0.8738, 1.0000, 0.8644, 0.8674, 0.8423, 0.8214,
         0.8104],
        [0.7943, 0.7995, 0.8189, 0.8763, 0.8644, 1.0000, 0.8475, 0.8412, 0.8272,
         0.8408],
        [0.7986, 0.7941, 0.8185, 0.8671, 0.8674, 0.8475, 1.0000, 0.8515, 0.8446,
         0.8165],
        [0.7680, 0.7534, 0.7997, 0.8386, 0.8423, 0.8412, 0.8515, 1.0000, 0.8348,
         0.8383],
        [0.8143, 0.8105, 0.8212, 0.8235, 0.8214, 0.8272, 0.8446, 0.8348, 1.000